In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import FunctionTransformer, RobustScaler, StandardScaler, OneHotEncoder, PowerTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import chi2_contingency
import matplotlib.ticker as mticker 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
customer_df = pd.read_csv('customer_data.csv')
payment_df = pd.read_csv('payment_data.csv') 

In [ ]:
customer_df.head()

In [ ]:
payment_df.head()

### payment_data.csv:
Lịch sử thanh toán thẻ của khách hàng.

- **id**: mã khách hàng
- **OVD_t1**: số lần quá hạn loại 1
- **OVD_t2**: số lần quá hạn loại 2
- **OVD_t3**: số lần quá hạn loại 3
- **OVD_sum**: tổng số ngày quá hạn
- **pay_normal**: số lần thanh toán bình thường
- **prod_code**: mã sản phẩm tín dụng
- **prod_limit**: hạn mức tín dụng của sản phẩm
- **update_date**: ngày cập nhật tài khoản
- **new_balance**: số dư hiện tại của sản phẩm
- **highest_balance**: số dư cao nhất trong lịch sử
- **report_date**: ngày thanh toán gần nhất

### customer_data.csv:
Dữ liệu nhân khẩu học và các thuộc tính danh mục của khách hàng đã được mã hóa.

- `fea_1`
- `fea_3`
- `fea_5`
- `fea_6`
- `fea_7`
- `fea_9`
- **label** là 1: khách hàng có rủi ro tín dụng cao
- **label** là 0: khách hàng có rủi ro tín dụng thấp

In [ ]:
df_full = pd.merge(customer_df, payment_df, on='id',how="inner")

In [ ]:
df_full.drop_duplicates(inplace=True)

In [ ]:
## Chuyển cột ngày tháng năm sang datetime
date_columns = ['update_date', 'report_date']
for col in date_columns:
    df_full[col] = pd.to_datetime(df_full[col], format='%d/%m/%Y', errors='coerce')

## 1. Quick glance at data

In [ ]:
df_full.describe().T.round()

In [ ]:
df_full.info()

In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mz_table = mz_table[mz_table.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        return mz_table
missing_values_table(df_full)

## 2. EDA cơ bản + xử lý dữ liệu

### Xem phân phối tổng quát

In [ ]:
# Xác định lại các cột số và cột phân loại
numerical_cols = [
    'fea_2', 'fea_4', 'fea_10','fea_8','fea_11',
    'OVD_t1', 'OVD_t2', 'OVD_t3', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]
categorical_cols = [
    'label', 'fea_1', 'fea_3', 'fea_5', 'fea_6', 'fea_7',  'fea_9', 
    'prod_code'
]

# Vẽ KDE plot cho các cột số (3 đồ thị trên 1 hàng)
print("Các cột numeric")
n_numerical = len(numerical_cols)
n_rows_numerical = math.ceil(n_numerical / 3)
fig_numerical, axes_numerical = plt.subplots(n_rows_numerical, 3, figsize=(18, n_rows_numerical * 5))
axes_numerical = axes_numerical.flatten()

for i, col in enumerate(numerical_cols):
    sns.kdeplot(data=df_full, x=col, fill=True, ax=axes_numerical[i])
    axes_numerical[i].set_title(f'Phân phối của cột: {col}')
    axes_numerical[i].set_xlabel(col)
    axes_numerical[i].set_ylabel('Mật độ')

# Ẩn các subplot không sử dụng
for j in range(i + 1, len(axes_numerical)):
    fig_numerical.delaxes(axes_numerical[j])

plt.tight_layout()
plt.show()

# Vẽ biểu đồ cột cho các cột phân loại (3 đồ thị trên 1 hàng)
print("Các cột category")
n_categorical = len(categorical_cols)
n_rows_categorical = math.ceil(n_categorical / 3)
fig_categorical, axes_categorical = plt.subplots(n_rows_categorical, 3, figsize=(18, n_rows_categorical * 5))
axes_categorical = axes_categorical.flatten()

for i, col in enumerate(categorical_cols):
    sns.countplot(data=df_full, x=col, ax=axes_categorical[i])
    axes_categorical[i].set_title(f'Phân phối của cột: {col}')
    axes_categorical[i].set_xlabel(col)
    axes_categorical[i].set_ylabel('Số lượng')

    # Sửa lỗi ở đây: Sử dụng set_xticklabels để xoay và căn chỉnh nhãn
    axes_categorical[i].set_xticklabels(axes_categorical[i].get_xticklabels(), rotation=45, ha='right')


# Ẩn các subplot không sử dụng
for j in range(i + 1, len(axes_categorical)):
    fig_categorical.delaxes(axes_categorical[j])

plt.tight_layout()
plt.show()

In [ ]:
# Xác định các cột số (bao gồm fea_8 và fea_10)
numerical_cols = [
    'fea_2', 'fea_4', 'fea_11', 'fea_10', 'fea_8',
    'OVD_t1', 'OVD_t2', 'OVD_t3', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Chọn chỉ các cột số từ DataFrame
df_numerical = df_full[numerical_cols]

# Tính ma trận tương quan
correlation_matrix = df_numerical.corr()

# Vẽ heatmap của ma trận tương quan
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap ma trận tương quan')
plt.show()


In [ ]:
# Xem xem có id nào bị gán nhãn là cả 0 và 1 không
df_full.groupby('id')["label"].nunique().pipe(lambda x: x[x > 1])

### Tìm hiểu những cột NULL + đề xuất phương án xử lý

### ***C. Tìm hiểu vấn đề về cột prod_limit***

Tạo biến chỉ báo prod_limit_is_missing 

In [ ]:
#Check xem nhóm rủi ro cao có tỷ lệ thiếu "prod_limit" cao hơn không
df_full['prod_limit_is_missing'] = df_full['prod_limit'].isnull().astype(int)

#### 1. Phân tích null của cột này

a. Giả thuyết: Null có nghĩa là "sản phẩm không có hạn mức tín dụng"

In [ ]:
# Tính tỷ lệ prod_limit bị thiếu cho mỗi prod_code và sắp xếp
prod_code_missing_ratio = df_full.groupby('prod_code')['prod_limit'].apply(lambda x: x.isnull().mean()*100).sort_values(ascending=False)

print("\nTỷ lệ prod_limit bị thiếu theo từng prod_code (sắp xếp từ cao đến thấp):")
print(prod_code_missing_ratio.head(20)) # In ra 20 dòng đầu để xem

# Vẽ biểu đồ cột
plt.figure(figsize=(15, 7)) # Điều chỉnh kích thước figure
prod_code_missing_ratio.plot(kind='bar')
plt.title('Tỷ lệ Prod_limit bị thiếu theo Prod_code')
plt.xlabel('Prod_code')
plt.ylabel('Tỷ lệ thiếu (%)')
plt.xticks(rotation=45, ha='right') # Xoay nhãn trục x để dễ đọc
plt.tight_layout() # Điều chỉnh layout
plt.show()


-> Tôi thấy một vài sản phẩm (ngoại trừ 22,10) có tỷ lệ null của cột prod_limit là 100%. Tuy nhiên các cột prod_code = 22, 10 thì lại có tỷ lệ không phải 100%. Chứng tỏ 2 cột này có tồn tại null -> Chứng tỏ null không đại diện cho "sản phẩm không có khái niệm giới hạn định mức" (bởi 22,10 vẫn bị giới hạn)

Tôi lại nghĩ đến giả thiết khác xảy ra là: với những giá trị null, nó ám chỉ đến việc sản phẩm này (tương ứng với người dùng nào đó sử dụng) chưa bị áp hạn mức do chưa vi phạm điều gì đó ?

b. Giả thuyết 2: Ban đầu, sản phẩm người dùng không bị giới hạn tín dụng (tức prod_limit = NULL). Nhưng sau đó, do vi phạm chính sách nào đó (hoặc có thể vì vấn đề nào đó khác) dẫn đến việc người đó bị prod_limit khác null (tức là lúc này người này bị giới hạn tín dụng)

Việc vi phạm chính sách dẫn đến prod_limit bị khác null (tức là lúc đó người dùng bị áp đặt hạn mức) có thể liên quan đến biến OVD_t1, OVD_t2, OVD_t3, OVD_sum

In [ ]:
print("\nMô tả các biến OVD và pay_normal khi prod_limit BỊ THIẾU:")
print(df_full[df_full['prod_limit'].isnull()][['OVD_t1', 'OVD_t2', 'OVD_t3', 'OVD_sum', 'pay_normal']].describe())

print("\nMô tả các biến OVD và pay_normal khi prod_limit KHÔNG BỊ THIẾU:")
print(df_full[df_full['prod_limit'].notnull()][['OVD_t1', 'OVD_t2', 'OVD_t3', 'OVD_sum', 'pay_normal']].describe())

##### -> Ở đây ta thấy được rằng: 
+ các giá trị mean tương ứng của các cột OVD khi prod_limit thiếu đều lớn hơn so với prod_limit không thiếu
+ mặt khác, pay_normal (chi trả bình thường) khi prod_limit thiếu thì mean lại có xu hướng ít hơn so với TH không bị thiếu (có thể là khi bị giới hạn tín dụng rồi, họ mới chi trả bthg. Còn đâu lúc ch bị, họ hay cheat)
+ Ở phần max cũng đa phần đều lớn hơn hoặc bằng  
##### -> Các điều trên đang củng cố giả thuyết của chúng ta

##### Ta tập trung vào các prod_code 22 và 10 do chúng có tồn tại những giá trị vừa null vừa 0 null ở cột prod_limit

In [ ]:
df_10_and_22 = df_full[(df_full['prod_code'] == 10) | (df_full['prod_code'] == 22)]

In [ ]:

# Nhóm theo id và prod_code, sau đó đếm số lượng bản ghi (không null) cho mỗi nhóm trong cột update_date
update_date_counts = df_10_and_22.groupby(["id","prod_code","update_date"])["prod_limit_is_missing"].value_counts().head(20)

print("Số lượng bản ghi theo ID và Prod_code và update_date (cho prod_limit_is_missing):")
print(update_date_counts)

-> Củng cố thêm giả thuyết của ta rằng: có khoảng thời gian, người dùng bị giối hạn, sau được gỡ, sau lại bị giới hạn

#### Vậy giá trị null ở đây có thể là các TH khả nghi như:
Khách hàng không đủ điều kiện để được cấp hạn mức tín dụng rõ ràng (có thể liên quan đến rủi ro của họ).Trong một số trường hợp, hạn mức có thể đã từng được áp dụng rồi lại được gỡ bỏ, cho thấy một quy trình quản lý hạn mức động

#### 2. kiểm tra xem cột prod_limit_is_missing mới này có ảnh hưởng đến target không

In [ ]:
# Giả sử các thư viện cần thiết (pandas, matplotlib.pyplot, seaborn) đã được import và df_full đã được tạo

# Lọc ra nhóm khách hàng có rủi ro thấp (label == 0)
low_risk_df = df_full[df_full['label'] == 0]

# Lọc ra nhóm khách hàng có rủi ro cao (label == 1)
high_risk_df = df_full[df_full['label'] == 1]

# Đếm số lượng cho từng trường hợp thiếu/không thiếu prod_limit trong nhóm rủi ro thấp
missing_limit_counts_low_risk = low_risk_df['prod_limit'].isnull().value_counts()

# Đếm số lượng cho từng trường hợp thiếu/không thiếu prod_limit trong nhóm rủi ro cao
missing_limit_counts_high_risk = high_risk_df['prod_limit'].isnull().value_counts()


# Tạo figure và các subplots (1 hàng, 2 cột)
fig, axes = plt.subplots(1, 2, figsize=(18, 9)) # Giữ kích thước figure

# Màu sắc tùy chỉnh (ví dụ: sử dụng bảng màu 'viridis' từ seaborn)
colors = sns.color_palette('viridis', 2) # Lấy 2 màu từ bảng màu viridis

# Nhãn cho biểu đồ tròn
pie_labels = ['Không thiếu Prod_limit', 'Thiếu Prod_limit']

# Vẽ biểu đồ tròn cho nhóm rủi ro thấp trên subplot đầu tiên
axes[0].pie(missing_limit_counts_low_risk,
            labels=pie_labels,
            autopct='%1.1f%%',
            startangle=90,
            colors=colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            textprops={'fontsize': 11, 'color': 'black', 'weight': 'bold'}) # Điều chỉnh màu và độ đậm của chữ

axes[0].set_title('Nhóm rủi ro thấp (Label = 0)', fontsize=14)
axes[0].axis('equal') # Đảm bảo biểu đồ tròn là hình tròn

# Vẽ biểu đồ tròn cho nhóm rủi ro cao trên subplot thứ hai
axes[1].pie(missing_limit_counts_high_risk,
            labels=pie_labels,
            autopct='%1.1f%%',
            startangle=90,
            colors=colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            textprops={'fontsize': 11, 'color': 'black', 'weight': 'bold'}) # Điều chỉnh màu và độ đậm của chữ

axes[1].set_title('Nhóm rủi ro cao (Label = 1)', fontsize=14)
axes[1].axis('equal') # Đảm bảo biểu đồ tròn là hình tròn

plt.tight_layout() # Điều chỉnh layout
plt.show()

Như tôi dự đoán ở trên nãy, do tôi vẫn nghĩ có thể khả năng cao null ở đây mang nghĩa không bị giới hạn tín dụng, còn khác null mang nghĩa bị giới hạn tín dụng (có thể do người dùng vi phạm chính sách thẻ dẫn đến rủi ro hay là tự đặt giới hạn cho thẻ khi nhận thấy rủi ro,...) thế nên ở đây prod_limit bị thiếu ở nhóm rủi ro thấp có xu hướng cao hơn prod_limit bị thiếu ở nhóm rủi ro cao. Tuy nhiên, như tôi nói, đây vẫn chỉ là giả định của tôi chứ thực tế ra sao chúng ta cần hỏi người cung cấp dữ liệu

2 biểu đồ tròn trên là khi ta so sánh trong TH: (id,lanbel,prod_code) bị lặp lại, tức là khi thẻ của 1 người nào đó bị xuất hiện prod_limit null nhiều lần (và cả không null cũng xuất hiện nhiều lần với thẻ đó) và trong tất cả TH thì người đó đều bị coi là label 1  
VD:
Khi biểu điễn bảng:   
id - prod_code - update_date - prod_limit - label  
1999 - 10 - 3/2 - null - 1      
1999 - 10 - 4/5 - null - 1    
1999 - 10 - 4/5 - không null - 1     
1999 - 10 - 5/5 - null - 1  
1999 - 10 - 6/5 - không null - 1  
-> Ta thấy vấn đề ở đây là một người bị xét là null quá nhiều lần, trong khi label người đó thì đã bị gán là 1. Vậy nên việc so sánh như trên là có vẻ chưa trung thực lắm (vì chỉ cần người đó bị gán label 1 cái là tất cả prod_limit dù null hay không null của người đó đều có label = 1)

Đề xuất xử lý: tạo 1 dataframe khác loại bỏ trùng ở các cột id, prod_limit_is_missing với mục tiêu: tránh để prod_limit xuất hiện nhiều lần trên cùng một người 

In [ ]:
df_remove_dup_id_prod = df_full.drop_duplicates(subset=["id", "prod_limit_is_missing"])

In [ ]:
# Giả sử các thư viện cần thiết (pandas, matplotlib.pyplot, seaborn) đã được import và df_full đã được tạo

# Lọc ra nhóm khách hàng có rủi ro thấp (label == 0)
low_risk_df = df_remove_dup_id_prod[df_remove_dup_id_prod['label'] == 0]

# Lọc ra nhóm khách hàng có rủi ro cao (label == 1)
high_risk_df = df_remove_dup_id_prod[df_remove_dup_id_prod['label'] == 1]

# Đếm số lượng cho từng trường hợp thiếu/không thiếu prod_limit trong nhóm rủi ro thấp
missing_limit_counts_low_risk = low_risk_df['prod_limit'].isnull().value_counts()

# Đếm số lượng cho từng trường hợp thiếu/không thiếu prod_limit trong nhóm rủi ro cao
missing_limit_counts_high_risk = high_risk_df['prod_limit'].isnull().value_counts()


# Tạo figure và các subplots (1 hàng, 2 cột)
fig, axes = plt.subplots(1, 2, figsize=(18, 9)) # Giữ kích thước figure

# Màu sắc tùy chỉnh (ví dụ: sử dụng bảng màu 'viridis' từ seaborn)
colors = sns.color_palette('viridis', 2) # Lấy 2 màu từ bảng màu viridis

# Nhãn cho biểu đồ tròn
pie_labels = ['Không thiếu Prod_limit', 'Thiếu Prod_limit']

# Vẽ biểu đồ tròn cho nhóm rủi ro thấp trên subplot đầu tiên
axes[0].pie(missing_limit_counts_low_risk,
            labels=pie_labels,
            autopct='%1.1f%%',
            startangle=90,
            colors=colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            textprops={'fontsize': 11, 'color': 'black', 'weight': 'bold'}) # Điều chỉnh màu và độ đậm của chữ

axes[0].set_title('Prod_limit trong nhóm rủi ro thấp (Label = 0)', fontsize=14)
axes[0].axis('equal') # Đảm bảo biểu đồ tròn là hình tròn

# Vẽ biểu đồ tròn cho nhóm rủi ro cao trên subplot thứ hai
axes[1].pie(missing_limit_counts_high_risk,
            labels=pie_labels,
            autopct='%1.1f%%',
            startangle=90,
            colors=colors,
            wedgeprops={'edgecolor': 'white', 'linewidth': 1.5},
            textprops={'fontsize': 11, 'color': 'black', 'weight': 'bold'}) # Điều chỉnh màu và độ đậm của chữ

axes[1].set_title('Prod_limit trong nhóm rủi ro cao (Label = 1)', fontsize=14)
axes[1].axis('equal') # Đảm bảo biểu đồ tròn là hình tròn

plt.tight_layout() # Điều chỉnh layout
plt.show()

Ở đây khách quan hơn, tuy vậy chúng ta vẫn thấy prod_limit thiếu ở label = 0 vãn lớn hơn label = 1. Và ở đây tôi nhận định rằng biến prod_limit_is_missing là có ý nghĩa trong việc dự đoán label

Phương án xử lý: 
+ Giữ prod_limit_is_missing
+ Thay các giá trị null của prod_limit bằng giá trị 0 với ý nghĩa: "khách hàng/sản phẩm đó không có hạn mức tín dụng" hoặc "hạn mức không áp dụng"

In [ ]:
df_full["prod_limit"] = df_full["prod_limit"].fillna(0)

### ***A. Phân tích và xử lý cột report_date***

#### 1. Phân tích sơ bộ

In [ ]:
# Khoảng thời gian bao phủ:
print(f"Ngày report_date sớm nhất: {df_full['report_date'].min()}")
print(f"Ngày report_date gần nhất: {df_full['report_date'].max()}")

#### Phán đoán nguyên nhân report_date bị thiếu:

1. Giả thuyết 1: Do chưa bao giờ giao dịch ?

In [ ]:
print(df_full[df_full['report_date'].isnull()][['OVD_t1', 'OVD_sum', 'pay_normal']].describe())

-> Sai vì nếu thế OVD_sum ít nhất phải bằng không

2. Do người thu thập dữ liệu không xác định được thời điểm giao dịch gần nhất ?

In [ ]:
# Tính tỷ lệ phần trăm giá trị NULL trong cột report_date, nhóm theo id
report_date_null_percentage_by_id = df_full.groupby('id')['report_date'].apply(lambda x: x.isnull().mean() * 100)

# Sắp xếp kết quả từ lớn nhất đến bé nhất
sorted_report_date_null_percentage_by_id = report_date_null_percentage_by_id.sort_values(ascending=False)

print("Tỷ lệ phần trăm Report Date bị thiếu \ntheo từng ID (sắp xếp từ cao đến thấp)\nlấy ra khoảng 20 người:")
print(sorted_report_date_null_percentage_by_id.head(20))


-> Từ đây ta thấy được là có vài người thì tỷ lệ null của cột đó là 100%, còn một vài người thì 75%, 66%,... Vậy nguyên nhân có thể đúng là do người thu thập có thể không xác định được những giao dịch gần đây hoặc chưa cập nhật những giao dịch gần đây vào trong dữ liệu nên nó mới bị null như vậy

#### Kiểm tra kỹ hơn

In [ ]:
# Tính bảng crosstab với tỷ lệ phần trăm theo hàng
crosstab_result = pd.crosstab(df_full['report_date'].isnull(), df_full['label'], normalize='index')

# Vẽ biểu đồ cột chồng
ax = crosstab_result.plot(kind='bar', stacked=True, figsize=(8, 6), color=['skyblue', 'lightcoral'])

plt.title('Tỷ lệ nhóm rủi ro theo trạng thái thiếu Report Date')
plt.xlabel('Report Date bị thiếu')
plt.ylabel('Tỷ lệ phần trăm')
plt.xticks(ticks=[0, 1], labels=['Không thiếu', 'Thiếu'], rotation=0) # Đặt nhãn trục x rõ ràng
plt.legend(title='Nhóm rủi ro', labels=['Thấp (0)', 'Cao (1)'])
plt.yticks([0, 0.2, 0.4, 0.6, 0.8, 1.0], ['0%', '20%', '40%', '60%', '80%', '100%']) # Định dạng trục y thành phần trăm
plt.grid(axis='y', linestyle='--', alpha=0.7) # Thêm lưới ngang
plt.tight_layout() # Điều chỉnh layout
plt.show()


-> Không có sự khác biệt giữa thiếu/không thiếu report_date đối với label, cho ta thấy được rằng việc thiếu có thể là ngẫu nhiên, không liên quan đến hành vi của khách hàng

Tuy nhiên, vì cột này thiếu khá nhiều (13.2%) nên việc tạo biến chỉ báo theo tôi vẫn là nên để mô hình có thể biết được đâu là giá trị ta đã thay null, đâu là giá trị ta chưa thay null

In [ ]:
df_full['report_date_is_missing'] = df_full['report_date'].isnull().astype(int)

#### Phương án xử lý NULL: tách cột này ra thành ngày/ tháng / năm và thay thế null bằng cách dùng KNN Imputer (sau khi chia tập dữ liệu)

In [ ]:
# Tách cột report_date thành ngày, tháng, năm
df_full['report_year'] = df_full['report_date'].dt.year
df_full['report_month'] = df_full['report_date'].dt.month
df_full['report_day'] = df_full['report_date'].dt.day
df_full.drop("report_date",axis=1,inplace=True)

### ***B. Tìm hiểu về cột update_date***

Cột này tuy trước đó người thực hiện đã phân tích, nhưng vẫn chưa hiểu rõ ý nghĩa thực sự của cột nên đã xóa đi phần phân tích đó. Vậy nên, người thực hiện xin phép không phân tích

Ta tập trung xem mấy giá trị null

In [ ]:

# Tạo biến chỉ báo
df_full['update_date_is_missing'] = df_full['update_date'].isnull().astype(int)
# Tính bảng chéo tỷ lệ phần trăm theo hàng (Risk Rate)
cross_tab_row_pct = pd.crosstab(df_full['update_date_is_missing'], df_full['label'], normalize='index') * 100

# Vẽ biểu đồ cột (không chồng)
ax1 = cross_tab_row_pct.plot(kind='bar', stacked=False, figsize=(8, 6))
plt.title('Tỷ lệ Label theo Trạng thái Thiếu/Đủ của Update Date')
plt.xlabel('Update Date Bị Thiếu') # Cập nhật nhãn trục x
plt.ylabel('Tỷ lệ Phần trăm (%)')
plt.xticks(rotation=0)
plt.legend(title='Label')
plt.tight_layout()
# Đổi nhãn trục x
ax1.set_xticklabels(['Không Thiếu', 'Thiếu'])
plt.show()

-> Chưa đủ để thấy sự khác biệt, mặt khác dữ liệu thiếu là ít (21 dữ liệu) nên ta quyết định bỏ

In [ ]:
df_full.dropna(subset=["update_date"],inplace=True)

In [ ]:
df_full.drop(["update_date_is_missing"],axis=1,inplace=True)

-> Tiếp tục chia cột này ra thành 3 cột day/month/year 

In [ ]:
# Giả sử cột 'update_date' đã là datetime

# Tách cột update_date thành ngày, tháng, năm
df_full['update_year'] = df_full['update_date'].dt.year
df_full['update_month'] = df_full['update_date'].dt.month
df_full['update_day'] = df_full['update_date'].dt.day

# Xóa cột update_date gốc
df_full.drop("update_date", axis=1, inplace=True)

### ***D. Xử lý null của cột highest_balance*** (Xử lý sau baseline)

-> Thay thế null của cột này bằng trung bình highest_balance tương ứng của mỗi người

In [ ]:
# Lấy ra các highest_balance mỗi người
df_subset_highest_balance = df_full[["id","highest_balance"]].drop_duplicates()
highest_balance_mean_person = df_subset_highest_balance.groupby('id')['highest_balance'].mean()

In [ ]:
highest_balance_mean_person

### ***E. Xử lý null của cột fea_2***

Do phân phối của fea_2 là phân phối gần chuẩn, mặt khác giá trị của fea_2 chỉ có giá trị nguyên   
-> ta sẽ thay giá trị cột fea_2 bằng median

### **Tóm lại ta có các phương án các cột chứa null như sau**:
1. prod_limit: thay null = 0, tạo 1 biến mới có tên prod_limit_is_missing
2. report_date: tách cột này thành 3 cột day/month/year, ta sẽ thay null = thuật toán KNN sau khi chia tập dữ liệu
3. fea_2: thay null = median của cột đó sau khi chia tập dữ liệu
4. update_date: loại bỏ những dòng null, chia cột đó thành 3 cột day/month/year
5. highest_balance: thay bằng giá trị trung bình của highest_balance tương ứng với từng id (group by id)


## Tiếp tục thực hiên EDA các biến khác

In [ ]:
# Xác định các cột số (bao gồm fea_8 và fea_10)
numerical_cols = [
    'fea_2', 'fea_4', 'fea_11', 'fea_10', 'fea_8',
    'OVD_t1', 'OVD_t2', 'OVD_t3', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Chọn chỉ các cột số từ DataFrame
df_numerical = df_full[numerical_cols]

# Tính ma trận tương quan
correlation_matrix = df_numerical.corr()

# Vẽ heatmap của ma trận tương quan
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap ma trận tương quan')
plt.show()


### Quyết định xử lý đa cộng tuyến giữa OVD_t3 và OVD_sum

-> Tồn tại đa cộng tuyến cao giữa OVD_t3 và OVD_sum, có khả năng cao 1 trong 2 biến này là thừa. Phương án xử lý khi đa cộng tuyến cao như vậy của tôi sẽ là bỏ 1 trong 2 biến

Quyết định bỏ biến OVD_t3 vì OVD_sum tổng quát hơn:

In [ ]:
df_full.drop("OVD_t3",axis=1,inplace=True)

In [ ]:
# Xác định các cột số (bao gồm fea_8 và fea_10)
numerical_cols = [
    'fea_2', 'fea_4', 'fea_11', 'fea_10', 'fea_8',
    'OVD_t1', 'OVD_t2',  'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Chọn chỉ các cột số từ DataFrame
df_numerical = df_full[numerical_cols]

# Tính ma trận tương quan
correlation_matrix = df_numerical.corr()

# Vẽ heatmap của ma trận tương quan
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap ma trận tương quan')
plt.show()

### Quyết định giữ lại biến new_balance và highest_balance (ko xử lý đa cộng tuyến)

Lý do:
+ new_balance: Phản ánh tình hình tài chính hiện tại của khách hàng tại thời điểm update_date (hoặc report_date gì đó). Đây là thông tin rất quan trọng và cập nhật.
+ highest_balance: Phản ánh mức số dư cao nhất trong lịch sử (tính đến update_date đó hoặc một thời điểm trước đó). Điều này có thể cho biết:

In [ ]:
# Xác định các cột số và thêm cột label vào danh sách
numerical_cols_with_label = [
    'label', # Thêm label vào danh sách
    'fea_2', 'fea_4', 'fea_11', 'fea_10', 'fea_8',
    'OVD_t1', 'OVD_t2', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Chọn chỉ các cột này từ DataFrame
df_subset = df_full[numerical_cols_with_label]

# Tính ma trận tương quan cho tập con này
correlation_matrix_subset = df_subset.corr()

# Lấy riêng dòng tương quan với label và sắp xếp
label_correlations = correlation_matrix_subset['label'].sort_values(ascending=False)

# Loại bỏ chính label khỏi kết quả
label_correlations = label_correlations.drop('label')

# Vẽ biểu đồ cột cho các hệ số tương quan
plt.figure(figsize=(10, 6))
label_correlations.plot(kind='bar', color='skyblue')
plt.title('Mối quan hệ giữa Label và các biến số')
plt.xlabel('Biến số')
plt.ylabel('Hệ số tương quan')
plt.axhline(y=0, color='black', linestyle='--')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

-> Ít có mối quan hệ tuyến tính, không phù hợp khi sử dụng các mô hình tuyến tính như logistic regression

## Triển khai mô hình baseline: Logistic Regression

1. Chia tập dữ liệu

In [ ]:
X = df_full.drop("label", axis=1)
y = df_full["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

2. Điền null

In [ ]:
# Thay null cột fea_2 bằng median do phân phối dạng chuẩn
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

imputer.fit(X_train[['fea_2']])

X_train['fea_2'] = imputer.transform(X_train[['fea_2']])
X_test['fea_2'] = imputer.transform(X_test[['fea_2']])

In [ ]:
# Xử lý null tại cột highest_balance
X_train['highest_balance_mean_by_id'] = X_train.groupby('id')['highest_balance'].transform('mean')

# Sử dụng .fillna() với cột trung bình theo id
X_train['highest_balance'] = X_train['highest_balance'].fillna(X_train['highest_balance_mean_by_id'])

# Bỏ cột trung bình tạm thời
X_train = X_train.drop(columns=['highest_balance_mean_by_id'])

mean_highest_balance_by_id_train = X_train.groupby('id')['highest_balance'].mean()

X_test['highest_balance_mean_by_id'] = X_test['id'].map(mean_highest_balance_by_id_train)

# Điền giá trị thiếu trong highest_balance của X_test
# Sử dụng .fillna() với cột trung bình theo id từ tập train
# Cần một chiến lược dự phòng nếu một id trong X_test không có trong tập train
# Ví dụ: điền bằng trung bình tổng thể của highest_balance từ X_train
overall_mean_highest_balance_train = X_train['highest_balance'].mean()
X_test['highest_balance'] = X_test['highest_balance'].fillna(X_test['highest_balance_mean_by_id']).fillna(overall_mean_highest_balance_train)

# Bỏ cột trung bình tạm thời
X_test = X_test.drop(columns=['highest_balance_mean_by_id'])

In [ ]:

# Xác định các cột số sẽ được sử dụng cho KNN imputation
# Bao gồm các cột cần điền và các cột số khác có thể giúp tìm hàng xóm
numerical_features_for_knn = [
    "id","prod_code",
    'report_day', 'report_month', 'report_year',
    'update_day', 'update_month', 'update_year', # Các cột ngày tháng đã tách từ update_date
    'OVD_t1', 'OVD_t2', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Chọn chỉ các cột số này từ X_train và X_test
X_train_numerical = X_train[numerical_features_for_knn]
X_test_numerical = X_test[numerical_features_for_knn]


# Khởi tạo KNNImputer
imputer = KNNImputer(n_neighbors=5) # Chọn số hàng xóm, bạn có thể điều chỉnh giá trị này

# Fit imputer CHỈ trên X_train_numerical và transform X_train_numerical
X_train_imputed = imputer.fit_transform(X_train_numerical)

# Transform X_test_numerical (KHÔNG FIT LẠI)
X_test_imputed = imputer.transform(X_test_numerical)

# Kết quả từ imputer là numpy array, cần gán lại vào DataFrame và giữ tên cột
X_train[numerical_features_for_knn] = X_train_imputed
X_test[numerical_features_for_knn] = X_test_imputed

# SAU KHI IMPUTE: Các giá trị có thể là số thực, cần làm tròn và chuyển về kiểu số nguyên
# Sử dụng .loc để tránh SettingWithCopyWarning nếu X_train/X_test là slice
for col in ['report_day', 'report_month', 'report_year']:
    X_train.loc[:, col] = X_train[col].round().astype('Int64') # Dùng Int64 để giữ NaN nếu có lỗi làm tròn
    X_test.loc[:, col] = X_test[col].round().astype('Int64') # Tương tự cho X_test

print("Đã điền giá trị thiếu cho các cột ngày/tháng/năm bằng KNN imputation.")

3. Xóa các cột thừa

In [ ]:
X_train.drop(["id","prod_code"],axis=1,inplace=True)
X_test.drop(["id","prod_code"],axis=1,inplace=True)

4. Mã hóa các cột category và chuẩn hóa các cột số

In [ ]:

numerical_cols = [
    'fea_2', 'fea_4', 'fea_11', 'fea_8', 'fea_10',
    'OVD_t1', 'OVD_t2', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

categorical_cols = [
    'fea_1', 'fea_3', 'fea_5', 'fea_6', 'fea_7', 'fea_9'
]

# Xác định các cột số có vẻ bị lệch nặng để áp dụng biến đổi Yeo-Johnson
skewed_cols = [
    'fea_4', 'fea_10',
    'OVD_t1', 'OVD_t2', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Các cột số còn lại không áp dụng biến đổi Yeo-Johnson
non_skewed_cols = [col for col in numerical_cols if col not in skewed_cols]

# Áp dụng biến đổi Yeo-Johnson cho các cột bị lệch trong X_train và X_test
yeo_johnson_transformer = PowerTransformer(method='yeo-johnson')

# Fit CHỈ trên X_train và transform cả X_train và X_test
X_train[skewed_cols] = yeo_johnson_transformer.fit_transform(X_train[skewed_cols])
X_test[skewed_cols] = yeo_johnson_transformer.transform(X_test[skewed_cols])

# Áp dụng StandardScaler cho TẤT CẢ các cột số trong X_train và X_test
scaler = StandardScaler()

# Fit CHỈ trên X_train và transform cả X_train và X_test
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


# --- Xử lý cột phân loại (One-Hot Encoding) ---

# Khởi tạo OneHotEncoder
# handle_unknown='ignore' để xử lý các danh mục mới trong tập test nếu có
# sparse_output=False để kết quả là dense array, dễ làm việc với DataFrame
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit CHỈ trên X_train và transform cả X_train và X_test
X_train_cat_encoded = ohe.fit_transform(X_train[categorical_cols])
X_test_cat_encoded = ohe.transform(X_test[categorical_cols])

# Tạo DataFrame từ kết quả One-Hot Encoding
# Lấy tên cột mới sau OHE
ohe_feature_names = ohe.get_feature_names_out(categorical_cols)
X_train_cat_df = pd.DataFrame(X_train_cat_encoded, index=X_train.index, columns=ohe_feature_names)
X_test_cat_df = pd.DataFrame(X_test_cat_encoded, index=X_test.index, columns=ohe_feature_names)

# Bỏ các cột phân loại gốc khỏi X_train và X_test
X_train = X_train.drop(columns=categorical_cols)
X_test = X_test.drop(columns=categorical_cols)

# Kết hợp các cột số đã xử lý và các cột phân loại đã mã hóa
X_train_processed = pd.concat([X_train, X_train_cat_df], axis=1)
X_test_processed = pd.concat([X_test, X_test_cat_df], axis=1)


In [ ]:
# Xác định lại danh sách các cột số (tên cột vẫn giữ nguyên sau tiền xử lý số)
numerical_cols = [
    'fea_2', 'fea_4', 'fea_11', 'fea_8', 'fea_10',
    'OVD_t1', 'OVD_t2', 'OVD_sum', 'pay_normal',
    'prod_limit', 'new_balance', 'highest_balance'
]

# Vẽ KDE plot cho các cột số (3 đồ thị trên 1 hàng)
print("Generating KDE plots for numerical columns in X_train_processed...")
n_numerical = len(numerical_cols)
n_rows_numerical = math.ceil(n_numerical / 3)
fig_numerical, axes_numerical = plt.subplots(n_rows_numerical, 3, figsize=(18, n_rows_numerical * 5))
axes_numerical = axes_numerical.flatten() # Làm phẳng mảng axes 2D thành 1D

for i, col in enumerate(numerical_cols):
    # Kiểm tra xem cột có tồn tại trong X_train_processed không trước khi vẽ
    if col in X_train_processed.columns:
        sns.kdeplot(data=X_train_processed, x=col, fill=True, ax=axes_numerical[i])
        axes_numerical[i].set_title(f'Phân phối của cột: {col} (Processed)')
        axes_numerical[i].set_xlabel(col)
        axes_numerical[i].set_ylabel('Mật độ')
    else:
        # Nếu cột không tồn tại (ví dụ: bị loại bỏ hoặc đổi tên trong quá trình xử lý), ẩn subplot
        fig_numerical.delaxes(axes_numerical[i])


# Ẩn các subplot không sử dụng nếu số cột không đủ lấp đầy hàng cuối
for j in range(i + 1, len(axes_numerical)):
    fig_numerical.delaxes(axes_numerical[j])

plt.tight_layout()
plt.show()

print("Plot generation complete.")


5. Train model

In [ ]:
# Giả sử X_train_processed và X_test_processed là dữ liệu đã qua các bước trên
model_baseline = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')
# class_weight='balanced' nếu dữ liệu mất cân bằng
model_baseline.fit(X_train_processed, y_train)

In [ ]:
# Dự đoán trên tập Train
y_pred_train_baseline = model_baseline.predict(X_train_processed)
y_pred_proba_train_baseline = model_baseline.predict_proba(X_train_processed)[:, 1] # Xác suất của lớp 1 cho tập train

# Dự đoán trên tập Test (bạn đã làm ở bước trước)
y_pred_test_baseline = model_baseline.predict(X_test_processed) # Hoặc y_pred_baseline từ bước trước
y_pred_proba_test_baseline = model_baseline.predict_proba(X_test_processed)[:, 1] # Hoặc y_pred_proba_baseline từ bước trước

6. So sánh hiệu suất

In [ ]:
# --- Đánh giá trên tập TRAIN ---
print("="*30 + " KẾT QUẢ TRÊN TẬP HUẤN LUYỆN (TRAIN) " + "="*30)
print("Confusion Matrix (Train):")
print(confusion_matrix(y_train, y_pred_train_baseline))
print("\nClassification Report (Train):")
print(classification_report(y_train, y_pred_train_baseline))

accuracy_train = accuracy_score(y_train, y_pred_train_baseline)
precision_train = precision_score(y_train, y_pred_train_baseline, zero_division=0) # zero_division=0 để tránh lỗi nếu không có TP+FP
recall_train = recall_score(y_train, y_pred_train_baseline, zero_division=0)
f1_train = f1_score(y_train, y_pred_train_baseline, zero_division=0)
roc_auc_train = roc_auc_score(y_train, y_pred_proba_train_baseline)

print(f"\nAccuracy (Train): {accuracy_train:.4f}")
print(f"Precision (Train - cho lớp 1): {precision_train:.4f}")
print(f"Recall (Train - cho lớp 1): {recall_train:.4f}")
print(f"F1-score (Train - cho lớp 1): {f1_train:.4f}")
print(f"ROC AUC (Train): {roc_auc_train:.4f}")
print("\n" + "="*80 + "\n")


# --- Đánh giá trên tập TEST ---
print("="*30 + " KẾT QUẢ TRÊN TẬP KIỂM TRA (TEST) " + "="*30)
print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_pred_test_baseline))
print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_test_baseline))

accuracy_test = accuracy_score(y_test, y_pred_test_baseline)
precision_test = precision_score(y_test, y_pred_test_baseline, zero_division=0)
recall_test = recall_score(y_test, y_pred_test_baseline, zero_division=0)
f1_test = f1_score(y_test, y_pred_test_baseline, zero_division=0)
roc_auc_test = roc_auc_score(y_test, y_pred_proba_test_baseline)

print(f"\nAccuracy (Test): {accuracy_test:.4f}")
print(f"Precision (Test - cho lớp 1): {precision_test:.4f}")
print(f"Recall (Test - cho lớp 1): {recall_test:.4f}")
print(f"F1-score (Test - cho lớp 1): {f1_test:.4f}")
print(f"ROC AUC (Test): {roc_auc_test:.4f}")
print("\n" + "="*80 + "\n")

# --- So sánh trực tiếp ---
print("="*30 + " SO SÁNH TRAIN vs TEST " + "="*30)
print(f"{'Metric':<15} | {'Train':<10} | {'Test':<10} | {'Difference (Train-Test)':<25}")
print("-"*70)
print(f"{'Accuracy':<15} | {accuracy_train:<10.4f} | {accuracy_test:<10.4f} | {accuracy_train - accuracy_test:<25.4f}")
print(f"{'Precision (1)':<15} | {precision_train:<10.4f} | {precision_test:<10.4f} | {precision_train - precision_test:<25.4f}")
print(f"{'Recall (1)':<15} | {recall_train:<10.4f} | {recall_test:<10.4f} | {recall_train - recall_test:<25.4f}")
print(f"{'F1-score (1)':<15} | {f1_train:<10.4f} | {f1_test:<10.4f} | {f1_train - f1_test:<25.4f}")
print(f"{'ROC AUC':<15} | {roc_auc_train:<10.4f} | {roc_auc_test:<10.4f} | {roc_auc_train - roc_auc_test:<25.4f}")

#### Đánh giá Mô hình Baseline và Đề xuất Các Bước Tiếp Theo

**Kết luận về Mô hình Baseline:**

*   Mô hình baseline hiện tại là một **điểm khởi đầu tốt**.
*   **Không có dấu hiệu overfitting rõ ràng:** Hiệu suất trên tập huấn luyện và tập kiểm tra khá tương đồng.
*   Mô hình có **khả năng phân loại nhất định** (ví dụ, ROC AUC ~0.73 - 0.75, cao hơn mức ngẫu nhiên 0.5).
*   **Thách thức chính:**
    *   **Precision cho lớp 1 (Rủi ro) còn rất thấp** (khoảng 0.29), dẫn đến nhiều dự đoán sai dương (False Positives).
    *   **F1-score cho lớp 1 cũng thấp** (khoảng 0.40) do ảnh hưởng của Precision thấp.
    *   Recall cho lớp 1 ở mức khá (khoảng 0.67 - 0.69), cho thấy mô hình bắt được một phần đáng kể các trường hợp rủi ro.
*   Vấn đề **mất cân bằng dữ liệu** (lớp 0 chiếm đa số) có thể là một trong những nguyên nhân chính ảnh hưởng đến Precision của lớp 1.

**Đề xuất cho các bước tiếp theo:**

Mục tiêu chính là cải thiện khả năng dự đoán cho lớp 1 (Rủi ro), đặc biệt là cân bằng giữa Precision và Recall tùy theo yêu cầu nghiệp vụ.

1.  **Xử lý Mất cân bằng Dữ liệu (Imbalanced Data) một cách Triệt để hơn:**
    *   **Phương pháp:**
        *   **Oversampling lớp thiểu số:** SMOTE, ADASYN.
        *   **Undersampling lớp đa số:** RandomUnderSampler, NearMiss, Tomek Links.
        *   **Kết hợp Oversampling và Undersampling:** Ví dụ, SMOTEENN, SMOTETomek.
    *   **Lưu ý quan trọng:** Các kỹ thuật này phải được áp dụng **CHỈ trên tập huấn luyện (`X_train`)** sau khi đã chia train/test để tránh data leakage.

2.  **Feature Engineering Sâu hơn:**
    *   **Tạo Feature Tương tác (Interaction Features):** Kết hợp các biến hiện có để tạo ra các feature mới có thể nắm bắt mối quan hệ phức tạp hơn (ví dụ: `fea_2 * prod_limit`, hoặc các tương tác dựa trên kiến thức nghiệp vụ).
    *   **Khai thác Biến Thời gian:**
        *   Tạo các feature về xu hướng (ví dụ: thay đổi số dư trong 3 tháng cuối, độ dốc của `OVD_sum` theo thời gian).
        *   Tạo các feature về tần suất (ví dụ: số lần cập nhật/thanh toán trong một khoảng thời gian).
        *   Tính toán các độ trễ, khoảng thời gian giữa các sự kiện quan trọng.
    *   **Xử lý Biến Categorical có nhiều giá trị:**
        *   **Target Encoding (Mean Encoding):** Thay thế category bằng giá trị trung bình của biến mục tiêu cho category đó (cẩn thận với overfitting, cần sử dụng trên tập train và áp dụng cho test, hoặc dùng cross-validation).
        *   **Frequency Encoding:** Thay thế category bằng tần suất xuất hiện của nó.
        *   **Gộp nhóm các category hiếm** thành một nhóm "Other".
    *   **Giải quyết Vấn đề Chất lượng Dữ liệu:**
        *   Nghiên cứu và đưa ra chiến lược xử lý cho các trường hợp có cùng `update_date` nhưng thông tin khác nhau cho cùng một (`id`, `prod_code`).

3.  **Lựa chọn Feature (Feature Selection):**
    *   Sau khi có nhiều feature hơn, sử dụng các kỹ thuật lựa chọn feature (ví dụ: dựa trên feature importance của mô hình, Recursive Feature Elimination - RFE, SelectKBest) để loại bỏ các feature nhiễu hoặc ít đóng góp, giúp mô hình tập trung vào các tín hiệu quan trọng.

4.  **Thử nghiệm các Mô hình Mạnh mẽ hơn:**
    *   **Random Forest:** Thường cho kết quả tốt và ít bị overfitting hơn Decision Tree đơn lẻ.
    *   **Gradient Boosting Machines (GBMs):**
        *   **XGBoost**
        *   **LightGBM** (thường nhanh hơn XGBoost và hiệu quả trên tập dữ liệu lớn)
        *   **CatBoost** (xử lý tốt biến categorical tự động)
    *   Các mô hình này có khả năng học các mối quan hệ phi tuyến và tương tác giữa các feature một cách hiệu quả.

5.  **Tinh chỉnh Ngưỡng Quyết định (Decision Threshold):**
    *   Đối với các mô hình phân loại nhị phân, thay vì sử dụng ngưỡng mặc định 0.5 trên `predict_proba`, hãy thử nghiệm các ngưỡng khác nhau.
    *   Sử dụng đường cong **Precision-Recall Curve** để tìm ngưỡng tối ưu hóa F1-score, hoặc một điểm cân bằng giữa Precision và Recall phù hợp với yêu cầu nghiệp vụ.

6.  **Tinh chỉnh Siêu tham số (Hyperparameter Tuning):**
    *   Sau khi đã chọn được một hoặc vài mô hình tiềm năng và có bộ feature tốt, sử dụng các kỹ thuật như GridSearchCV, RandomizedSearchCV, hoặc các thư viện tối ưu hóa siêu tham số (ví dụ: Optuna, Hyperopt) để tìm bộ siêu tham số tốt nhất cho mô hình.

Bằng cách thực hiện các bước này một cách có hệ thống, bạn có khả năng cao sẽ cải thiện đáng kể hiệu suất của mô hình so với baseline hiện tại.

## Triển khai mô hình thứ 2: RandomForest

In [ ]:
# Khởi tạo mô hình Random Forest (có thể dùng cùng tham số như Cách 1 để so sánh công bằng)
rf_model_v2 = RandomForestClassifier(random_state=42, 
                                     n_estimators=100, 
                                     class_weight='balanced_subsample' if y_train.value_counts(normalize=True)[1] < 0.4 else None)

# Huấn luyện mô hình
rf_model_v2.fit(X_train_processed, y_train) # Sử dụng X_train_c2 đã qua xử lý kỹ lưỡng

# Dự đoán trên tập Train và Test
y_pred_train_v2 = rf_model_v2.predict(X_train_processed)
y_pred_proba_train_v2 = rf_model_v2.predict_proba(X_train_processed)[:, 1]

y_pred_test_v2 = rf_model_v2.predict(X_test_processed)
y_pred_proba_test_v2 = rf_model_v2.predict_proba(X_test_processed)[:, 1]

In [ ]:
# --- Đánh giá trên tập TRAIN ---
print("="*30 + " KẾT QUẢ TRÊN TẬP HUẤN LUYỆN (TRAIN) " + "="*30)
print("Confusion Matrix (Train):")
print(confusion_matrix(y_train, y_pred_train_v2))
print("\nClassification Report (Train):")
print(classification_report(y_train, y_pred_train_v2))

accuracy_train = accuracy_score(y_train, y_pred_train_v2)
precision_train = precision_score(y_train, y_pred_train_v2, zero_division=0) # zero_division=0 để tránh lỗi nếu không có TP+FP
recall_train = recall_score(y_train, y_pred_train_v2, zero_division=0)
f1_train = f1_score(y_train, y_pred_train_v2, zero_division=0)
roc_auc_train = roc_auc_score(y_train, y_pred_proba_train_v2)

print(f"\nAccuracy (Train): {accuracy_train:.4f}")
print(f"Precision (Train - cho lớp 1): {precision_train:.4f}")
print(f"Recall (Train - cho lớp 1): {recall_train:.4f}")
print(f"F1-score (Train - cho lớp 1): {f1_train:.4f}")
print(f"ROC AUC (Train): {roc_auc_train:.4f}")
print("\n" + "="*80 + "\n")


# --- Đánh giá trên tập TEST ---
print("="*30 + " KẾT QUẢ TRÊN TẬP KIỂM TRA (TEST) " + "="*30)
print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_pred_test_v2))
print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_test_v2))

accuracy_test = accuracy_score(y_test, y_pred_test_v2)
precision_test = precision_score(y_test, y_pred_test_v2, zero_division=0)
recall_test = recall_score(y_test, y_pred_test_v2, zero_division=0)
f1_test = f1_score(y_test, y_pred_test_v2, zero_division=0)
roc_auc_test = roc_auc_score(y_test, y_pred_proba_test_v2)

print(f"\nAccuracy (Test): {accuracy_test:.4f}")
print(f"Precision (Test - cho lớp 1): {precision_test:.4f}")
print(f"Recall (Test - cho lớp 1): {recall_test:.4f}")
print(f"F1-score (Test - cho lớp 1): {f1_test:.4f}")
print(f"ROC AUC (Test): {roc_auc_test:.4f}")
print("\n" + "="*80 + "\n")

# --- So sánh trực tiếp ---
print("="*30 + " SO SÁNH TRAIN vs TEST " + "="*30)
print(f"{'Metric':<15} | {'Train':<10} | {'Test':<10} | {'Difference (Train-Test)':<25}")
print("-"*70)
print(f"{'Accuracy':<15} | {accuracy_train:<10.4f} | {accuracy_test:<10.4f} | {accuracy_train - accuracy_test:<25.4f}")
print(f"{'Precision (1)':<15} | {precision_train:<10.4f} | {precision_test:<10.4f} | {precision_train - precision_test:<25.4f}")
print(f"{'Recall (1)':<15} | {recall_train:<10.4f} | {recall_test:<10.4f} | {recall_train - recall_test:<25.4f}")
print(f"{'F1-score (1)':<15} | {f1_train:<10.4f} | {f1_test:<10.4f} | {f1_train - f1_test:<25.4f}")
print(f"{'ROC AUC':<15} | {roc_auc_train:<10.4f} | {roc_auc_test:<10.4f} | {roc_auc_train - roc_auc_test:<25.4f}")


-> Mặc dù con số trên tập test là khá ấn tượng, xong ta thấy hiện tượng overfitting nặng

#### Phương án xử lý: tìm bộ siêu tham số giúp cân bằng giữa train và test bằng cách áp dụng RandomizedSearchCV

In [ ]:
from scipy.stats import randint
# Định nghĩa không gian siêu tham số để tìm kiếm
param_distributions = {
    'n_estimators': randint(100, 500),  # Số cây từ 100 đến 499
    'max_depth': [5, 8, 10, 15, 20, 25, 30, None], # Vẫn có thể dùng danh sách cho một số tham số
    'min_samples_split': randint(2, 21), # Số mẫu tối thiểu để chia từ 2 đến 20
    'min_samples_leaf': randint(1, 21),  # Số mẫu tối thiểu ở lá từ 1 đến 20
    'max_features': ['sqrt', 'log2', 0.3, 0.5, 0.7], # Có thể dùng danh sách hoặc phân phối
    'class_weight': [None, 'balanced', 'balanced_subsample'],
    'bootstrap': [True, False] # Thêm một tham số ví dụ
}



rf_clf = RandomForestClassifier(random_state=42,oob_score=True)

# Sử dụng GridSearchCV
random_search = RandomizedSearchCV(estimator=rf_clf,
                                   param_distributions=param_distributions,
                                   n_iter=50,  # Thử 50 tổ hợp ngẫu nhiên (bạn có thể điều chỉnh)
                                   cv=3,
                                   verbose=2,
                                   random_state=42,
                                   n_jobs=-1,
                                   scoring='roc_auc') 

In [ ]:
# Fit RandomizedSearchCV trên dữ liệu huấn luyện
print("Bắt đầu RandomizedSearchCV...")
random_search.fit(X_train_processed, y_train)
print("RandomizedSearchCV hoàn thành.")

In [ ]:
# In ra các siêu tham số tốt nhất
print("\nSiêu tham số tốt nhất tìm được từ RandomizedSearchCV:")
print(random_search.best_params_)

# Lấy mô hình tốt nhất
best_rf_model_random = random_search.best_estimator_

In [ ]:
# Đánh giá mô hình tốt nhất trên tập train và test
y_pred_train_best_random = best_rf_model_random.predict(X_train_processed)
y_pred_proba_train_best_random = best_rf_model_random.predict_proba(X_train_processed)[:, 1]

y_pred_test_best_random = best_rf_model_random.predict(X_test_processed)
y_pred_proba_test_best_random = best_rf_model_random.predict_proba(X_test_processed)[:, 1]

In [ ]:
# --- Đánh giá trên tập TRAIN ---
print("="*30 + " KẾT QUẢ TRÊN TẬP HUẤN LUYỆN (TRAIN) " + "="*30)
print("Confusion Matrix (Train):")
print(confusion_matrix(y_train, y_pred_train_best_random))
print("\nClassification Report (Train):")
print(classification_report(y_train, y_pred_train_best_random))

accuracy_train = accuracy_score(y_train, y_pred_train_best_random)
precision_train = precision_score(y_train, y_pred_train_best_random, zero_division=0) # zero_division=0 để tránh lỗi nếu không có TP+FP
recall_train = recall_score(y_train, y_pred_train_best_random, zero_division=0)
f1_train = f1_score(y_train, y_pred_train_best_random, zero_division=0)
roc_auc_train = roc_auc_score(y_train, y_pred_proba_train_best_random)

print(f"\nAccuracy (Train): {accuracy_train:.4f}")
print(f"Precision (Train - cho lớp 1): {precision_train:.4f}")
print(f"Recall (Train - cho lớp 1): {recall_train:.4f}")
print(f"F1-score (Train - cho lớp 1): {f1_train:.4f}")
print(f"ROC AUC (Train): {roc_auc_train:.4f}")
print("\n" + "="*80 + "\n")


# --- Đánh giá trên tập TEST ---
print("="*30 + " KẾT QUẢ TRÊN TẬP KIỂM TRA (TEST) " + "="*30)
print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_pred_test_best_random))
print("\nClassification Report (Test):")
print(classification_report(y_test, y_pred_test_best_random))

accuracy_test = accuracy_score(y_test, y_pred_test_best_random)
precision_test = precision_score(y_test, y_pred_test_best_random, zero_division=0)
recall_test = recall_score(y_test, y_pred_test_best_random, zero_division=0)
f1_test = f1_score(y_test, y_pred_test_best_random, zero_division=0)
roc_auc_test = roc_auc_score(y_test, y_pred_proba_test_best_random)

print(f"\nAccuracy (Test): {accuracy_test:.4f}")
print(f"Precision (Test - cho lớp 1): {precision_test:.4f}")
print(f"Recall (Test - cho lớp 1): {recall_test:.4f}")
print(f"F1-score (Test - cho lớp 1): {f1_test:.4f}")
print(f"ROC AUC (Test): {roc_auc_test:.4f}")
print(f"OOB score: {best_rf_model_random.o}")
print("\n" + "="*80 + "\n")

# --- So sánh trực tiếp ---
print("="*30 + " SO SÁNH TRAIN vs TEST " + "="*30)
print(f"{'Metric':<15} | {'Train':<10} | {'Test':<10} | {'Difference (Train-Test)':<25}")
print("-"*70)
print(f"{'Accuracy':<15} | {accuracy_train:<10.4f} | {accuracy_test:<10.4f} | {accuracy_train - accuracy_test:<25.4f}")
print(f"{'Precision (1)':<15} | {precision_train:<10.4f} | {precision_test:<10.4f} | {precision_train - precision_test:<25.4f}")
print(f"{'Recall (1)':<15} | {recall_train:<10.4f} | {recall_test:<10.4f} | {recall_train - recall_test:<25.4f}")
print(f"{'F1-score (1)':<15} | {f1_train:<10.4f} | {f1_test:<10.4f} | {f1_train - f1_test:<25.4f}")
print(f"{'ROC AUC':<15} | {roc_auc_train:<10.4f} | {roc_auc_test:<10.4f} | {roc_auc_train - roc_auc_test:<25.4f}")


In [ ]:

try:
    feature_names = X_train_processed.columns
except AttributeError:
    # Nếu X_train_processed là numpy array, bạn cần có danh sách tên feature từ trước
    # Ví dụ: feature_names = list_of_your_feature_names_in_order
    print("X_train_processed không phải là DataFrame. Hãy đảm bảo bạn có danh sách feature_names chính xác.")
    # Dừng lại hoặc cung cấp feature_names thủ công
    # feature_names = [...] # Điền tên feature của bạn vào đây theo đúng thứ tự

# Lấy giá trị feature importances từ mô hình
importances = best_rf_model_random.feature_importances_

# Tạo một Series Pandas để dễ dàng xem và sắp xếp
forest_importances = pd.Series(importances, index=feature_names)

# Sắp xếp các feature theo tầm quan trọng giảm dần
sorted_forest_importances = forest_importances.sort_values(ascending=False)

# In ra các feature quan trọng nhất (ví dụ: top 20)
print("Top 20 Feature Importances từ Random Forest:")
print(sorted_forest_importances.head(20))

# Trực quan hóa Feature Importances (ví dụ: top 20)
plt.figure(figsize=(10, 8)) # Điều chỉnh kích thước nếu cần
top_n = 20
sns.barplot(x=sorted_forest_importances.head(top_n).values, y=sorted_forest_importances.head(top_n).index)
# Hoặc dùng pandas plot:
# sorted_forest_importances.head(top_n).plot(kind='barh')
plt.title(f'Top {top_n} Feature Importances từ Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.gca().invert_yaxis() # Để feature quan trọng nhất ở trên cùng nếu dùng barh
plt.tight_layout()
plt.show()

-> Không có feature nào vượt trội, khá tương đồng nhau. Cho thấy mô hình đang học từ nhiều khía cạnh của dữ liệu